# 主程式

In [11]:
# update anytime
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# 讀取套件與函數
from t0_config import *
from library.apply import is_english_start, split_string_into_chunks, translation_filter
from transformers import pipeline
from docx import Document
from tqdm import tqdm, trange
import os
os.chdir(doc_path)

In [13]:
# 讀取Word檔案
doc = Document(file_name + '.docx')
raw_text_list = [paragraph.text for paragraph in doc.paragraphs]
target_text_list = translation_filter(raw_text_list, target_range)

即將翻譯第 0 句至第 29 句
從以下開始翻譯: 车辆格栅结构


In [4]:
# 讀取機器學習模型
translator = pipeline("translation", model=model_checkpoint)

model.safetensors: 100%|██████████| 310M/310M [00:37<00:00, 8.26MB/s] 


In [6]:
# 開始一句句翻譯
sentence_max_length = 250
trans_text_list = []
for sentence in tqdm(target_text_list):
    # 空白/英文開頭的段落不翻譯
    if is_english_start(sentence):
        trans_text_list.append(sentence)
        continue

    # 如果段落長度小於最大長度，直接翻譯，否則切完再翻譯
    if len(sentence) < sentence_max_length:
        paragraph = translator(sentence, max_length=1000, return_text=True)[0]['translation_text']
    else:
        temp_text_list = split_string_into_chunks(sentence, sentence_max_length)
        paragraph = ''
        for temp_text in tqdm(temp_text_list):
            paragraph += translator(temp_text, max_length=1000, return_text=True)[0]['translation_text']
    trans_text_list.append(paragraph)

100%|██████████| 91/91 [05:58<00:00,  3.94s/it]


In [8]:
# 產生新的Word檔案
for i in trange(len(trans_text_list)):
    paragraph = doc.paragraphs[i]
    paragraph.text = trans_text_list[i]

    # 如果格式是標題2，加粗
    style = paragraph.style.name
    if style == 'Heading 2':
        paragraph.style.font.bold = True

# 儲存成新檔案
doc.save(file_name + '_trans' + '.docx')

100%|██████████| 91/91 [00:00<00:00, 442.26it/s]


# 其他

In [ ]:
translator_raw = pipeline("translation", model='Helsinki-NLP/opus-mt-zh-en', tokenizer=model_checkpoint)